In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [112]:
#計測地点のosm_idを指定
start_osm = 228330073
mid_osm = [228330071,38135912,52171227,52171226,52171225]
end_osm = 315917801
start_mesh = 50325643134
end_mesh = 50325623342

In [113]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [114]:

sttime = "2022-04-01 12:00:00"
dbname = "kddi_trkn12_change_f2_22"

In [115]:
datedir = "230213"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_12_param  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値65.0距離0.0 km
50325643123
存在メッシュ：50325643123値71.95155150494391距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値65.0距離0.0 km
50325643141
存在メッシュ：50325643141値69.72129174667516距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値67.94580569706059距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値74.5766953391881距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値71.95155150494391距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値77.3473730477406距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値81.47416493629078距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値80.18412522556673距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値78.90314993873241距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値84.15343817104821距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値81.47416493629078距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値80.18412522556673距離0.5955503728502

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値68.0距離0.0 km
50325643123
存在メッシュ：50325643123値74.06032695302802距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値68.0距離0.0 km
50325643141
存在メッシュ：50325643141値72.11599793299885距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値70.56813830000154距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値76.34891388544604距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値74.06032695302802距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値78.76437650315847距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値82.36209250856119距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値81.23744250434022距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値80.1206948183821距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値84.69786917475999距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値82.36209250856119距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値81.23744250434022距離0.595550372850

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値69.0距離0.0 km
50325643123
存在メッシュ：50325643123値76.12979641532709距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値69.0距離0.0 km
50325643141
存在メッシュ：50325643141値73.84235050941042距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値72.0213391764724距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値78.82225162993652距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値76.12979641532709距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値81.66397235665704距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値85.89657942183669距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値84.57346176981203距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値83.25964096280248距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値88.64455197030586距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値85.89657942183669距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値84.57346176981203距離0.595550372850

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値71.0距離0.0 km
50325643123
存在メッシュ：50325643123値77.59506168417755距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値71.0距離0.0 km
50325643141
存在メッシュ：50325643141値75.47917422120464距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値73.79473873823697距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値80.08558275769128距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値77.59506168417755距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値82.71417442990776距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値86.62933596519895距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値85.40545213707613距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値84.19016789059229距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値89.17121057253291距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値86.62933596519895距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値85.40545213707613距離0.59555037285

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値59.0距離0.0 km
50325643123
存在メッシュ：50325643123値63.45612275957943距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値59.0距離0.0 km
50325643141
存在メッシュ：50325643141値62.026469068381516距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値60.88833698529525距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値65.13890726871033距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値63.45612275957943距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値66.91498272291065距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値69.56036213864793距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値68.73341360613252距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値67.91227560175155距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値71.27784498144116距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値69.56036213864793距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値68.73341360613252距離0.5955503728

...

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値62.0距離0.0 km
50325643123
存在メッシュ：50325643123値70.37751078800933距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値62.0距離0.0 km
50325643141
存在メッシュ：50325643141値67.68976184855724距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値65.55007353235507距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値73.5411456651754距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値70.37751078800933距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値76.88016751907202距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値81.85348082065812距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値80.29881757952913距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値78.7550781312929距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値85.08234856510938距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値81.85348082065812距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値80.29881757952913距離0.5955503728502

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値77.0距離0.0 km
50325643123
存在メッシュ：50325643123値85.02102096724298距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値77.0距離0.0 km
50325643141
存在メッシュ：50325643141値82.44764432308672距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値80.39900657353145距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値88.05003308367858距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値85.02102096724298距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値91.24696890123917距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値96.00865184956629距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値94.52014449103854距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値93.04209608315279距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値99.10012096659409距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値96.00865184956629距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値94.52014449103854距離0.59555037285

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
存在メッシュ：50325643134値56.0距離0.0 km
50325643123
存在メッシュ：50325643123値58.49542874536448距離0.2726531182505025 km
50325643134
存在メッシュ：50325643134値56.0距離0.0 km
50325643141
50325643141 is kddi none!
50325643132
存在メッシュ：50325643132値57.05746871176534距離0.11554012201340155 km
50325643121
50325643121 is kddi none!
50325643123
存在メッシュ：50325643123値58.49542874536448距離0.2726531182505025 km
50325633343
50325633343 is kddi none!
50325633342
存在メッシュ：50325633342値61.913802797642845距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値61.45071161943421距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値60.990874336980866距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値62.87559318960705距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値61.913802797642845距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値61.45071161943421距離0.595550372850224 km
50325633324
存在メッシュ：50325633324値62.87559318960705距離0.7512344026855676 km
50325633322
50325633

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
50325643134 is kddi none!
50325643123
存在メッシュ：50325643123値89.5557556983925距離0.2726531182505025 km
50325643134
50325643134 is kddi none!
50325643141
存在メッシュ：50325643141値86.8108206112925距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値84.62560701176687距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値92.78670195592382距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値89.5557556983925距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値96.19676682798844距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値101.27589530620403距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値99.68815412377444距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値98.11156915536297距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値104.57346236436703距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値101.27589530620403距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値99.68815412377444距離0.595550372850224 km
5032

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134
50325643134 is kddi none!
50325643123
存在メッシュ：50325643123値79.30804132571026距離0.2726531182505025 km
50325643134
50325643134 is kddi none!
50325643141
存在メッシュ：50325643141値76.96340927214568距離0.18517807369848877 km
50325643132
存在メッシュ：50325643132値75.09687265588421距離0.11554012201340155 km
50325643121
存在メッシュ：50325643121値82.06780792068493距離0.3756162699661618 km
50325643123
存在メッシュ：50325643123値79.30804132571026距離0.2726531182505025 km
50325633343
存在メッシュ：50325633343値84.98057166557346距離0.4842875380085171 km
50325633342
存在メッシュ：50325633342値89.31899390738262距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値87.96279831405732距離0.595550372850224 km
50325633344
存在メッシュ：50325633344値86.61613198687253距離0.5453080771398616 km
50325633324
存在メッシュ：50325633324値92.1356657695635距離0.7512344026855676 km
50325633342
存在メッシュ：50325633342値89.31899390738262距離0.6461481925665478 km
50325633341
存在メッシュ：50325633341値87.96279831405732距離0.595550372850224 km
50325

In [116]:
connection.close()

In [ ]:
#CreateGeojsonFile